In [8]:
import torch
import cv2
import numpy as np
import pandas as pd
import time

In [3]:
model = torch.hub.load("ultralytics/yolov5", "custom", path="./best.pt")

Using cache found in C:\Users\spec3/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-25 Python-3.9.15 torch-1.12.1 CUDA:0 (NVIDIA GeForce RTX 4080, 16376MiB)

Fusing layers... 
Model summary: 267 layers, 46129818 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 


In [9]:
#cap = cv2.VideoCapture(0) # webcam or connected cam
PATH = r"C:\Users\spec3\Downloads\켈리클락슨을 당황시킨 방탄 미국 팬들 함성.mp4"
cap = cv2.VideoCapture(PATH) # Loaded video

thr = 0.5 # 검출 정확도
while cap.isOpened():
    ret, frame = cap.read()
    
    results = model(frame)
    cv2.imshow("Detection", np.squeeze(results.render()))
    # 라벨 출력
    labels = results.pandas().xyxy[0]
    detect_labels = labels[labels["confidence"]>thr]["name"]
    print(detect_labels.tolist())
    
    if cv2.waitKey(10)&0xFF==ord("q"): break
    
cap.release()
cv2.destroyAllWindows()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Anger']
['Anger']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Anger']
['Anger']
['Anger']
['Anger']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Anger']
[]
[]
['Anger']
['Anger']
['Anger']
[]
['Anger']
[]
[]
[]
[]
['Anger']
['Anger']
['Anger']
['Anger']
['Anger']
[]
['Anger']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Panic/Anxiety']
['Panic/Anxiety']
['Panic/Anxiety']
['Panic/Anxiety']
['Panic/Anxiety']
['Panic/Anxiety']
['Panic/Anxiety']
[]
[]
['Panic/Anxiety']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Panic/Anxiety']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]